In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from pymongo import MongoClient
import matplotlib.pyplot as plt
%matplotlib inline

### Page display simulation

In [2]:
# Base URL of the site to be analyzed
SITE_URL = 'https://alianzaverde.org.co'

In [3]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [4]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\JCristancho\AppData\Local\Temp\ipykernel_12800\1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [5]:
def make_request(browser,relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL+relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(5)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [6]:
 soup = make_request(browser,'/lideres-verdes/congreso-2022/senado-2022')


### Extract and organize the information of each of the candidates

In [46]:
# Get HTML content for candidate listing page
soup = make_request(browser,'/lideres-verdes/congreso-2022/senado-2022')
# Find the section where the data of each of the candidates is contained 
candidates = soup.find_all(class_ = 'sppb-col-md-3')
print(len(candidates))        

36


In [47]:
# Building a list with the url of the photo, the description and social networks of each of the candidates
news = []
socialmedia = []
i = 0
for candidate in candidates:
    i+=1
    socialMediaLinks = ''
    if candidate.find(class_ = 'sppb-addon-wrapper')!= None:
        if candidate.find('ul',class_ = 'sppb-person-social')!= None:
            socialmedia = candidate.find('ul',class_ = 'sppb-person-social').find_all('li')
            for x in socialmedia:
                socialMediaLinks = socialMediaLinks+' ---- '+ x.find('a')['href']
            news.append({ 
            'Candidate number': i ,
            'Photo': 'https://alianzaverde.org.co/'+format (candidate.find('div',class_ = 'sppb-person-image').find(class_='sppb-img-responsive').get('src')),
            'Description': candidate.find(class_ = 'person-content-hover-content-wrap').find('span').get_text(),
            'Social media': socialMediaLinks
            })
        else:
            news.append({
            'Candidate number': i ,
            'Photo': 'https://alianzaverde.org.co/'+format(candidate.find('div',class_ = 'sppb-person-image').get('src')),
            'Description': candidate.find(class_ = 'person-content-hover-content-wrap').find('span').get_text(),
            'Social media': 'Does not have'
            })
        

In [48]:
news

[{'Candidate number': 1,
  'Photo': 'https://alianzaverde.org.co//images/Candidatos_2022/senado/2.jpg',
  'Description': 'Abogada y filósofa, con un máster en Ciencias de la Administración Pública, en Alemania. Experta en gestión pública, descentralización, transparencia, buen gobierno, modernización del estado, cultura ciudadana y cooperación internacional, temas en los cuales se ha desempeñado alternativamente como servidora pública, consultora y docente.',
  'Social media': ' ---- https://twitter.com/VivianaBarbere  ---- https://twitter.com/VivianaBarbere ---- https://www.instagram.com/barberenaviviana/ '},
 {'Candidate number': 2,
  'Photo': 'https://alianzaverde.org.co//images/Candidatos_2022/senado/3.jpg',
  'Description': 'Nacida de seres excepcionales, en tiempos inciertos; es la mujer de retos resueltos. Desde el inicio de su carrera política, su liderazgo se enfoca en mantener la cultura de la equidad, la familia y el fortalecimiento económico, político y social de las region